# Aerodynamic Drag 1 - Components of Drag

This notebook explores how we can breakdown the complex process of drag estimation, into more manageable actions.

## Coarse Formulation of Drag

The force of drag acting on an aircraft can be broken down into linear contributions from many sources

$$D = \Sigma_{j=0}^{N} D_j \qquad \text{with} \qquad N >> 1$$

It's helpful to non-dimensionalise the drag force into a linear sum of coefficients of drag

$$D = \frac{1}{2}\rho V^2 S C_D \qquad \text{where} \qquad C_D = \Sigma_{j=0}^N C_{D_{j}}$$

and now, we can break up this drag coefficient $C_D$ into one of the following categories

\begin{eqnarray}
C_D &=& C_{D_0} &+& C_{D_i} &+& C_{D_w} \\
\uparrow && \uparrow && \uparrow && \uparrow \\
\text{Total Drag} &=& \text{Profile Drag} &+& \text{Induced Drag} &+& \text{Wave Drag}
\end{eqnarray}

We'll explore contributions to these categories momentarily, but it's important to understand that most contributions to these categories can themselves be broken into two subcategories for the purposes of effectual mathematical modelling:
    
1. **Drag due to Lift**:
    These are components of drag that arise as a result of a vehicle needing to produce a lift force. If an aerodynamic body is not producing lift, the contributions from these components are negligible.

1. **Zero-lift drag**:
    What doesn't fall into the subcategory of lift-dependent drag are simply those drag subcomponents that are independent of lift. The magnitude of non-dimensional drag coefficients is non-zero and contribute to $C_{D}$ - even when the aerodynamic object is producing no lift.

In most introductory lessons to drag, you'll find slightly different representations of the drag breakdown using similar symbology and scripting.
We'll present two common portrayals below, in terms of the components described above.

##### Drag Model 1

The first model maintains the components of our original drag formulation, and all components are functions of lift:

\begin{eqnarray}
\text{Profile Drag } (C_{D_0}) &=& \text{Zero-lift drag components} &+& \text{(Lift-) Induced drag components}\\
\text{Induced Drag } (C_{D_i}) &=& 0 &+& \text{(Lift-) Induced drag components}\\
\text{Wave Drag } (C_{D_w}) &=& \text{Zero-lift drag components} &+& \text{(Lift-) Induced drag components}
\end{eqnarray}

##### Drag Model 2

In the second model, components of drag are broken down into those independent of lift (*zero-lift*) and dependent on lift (*lift-induced*):

\begin{eqnarray}
\text{Minimum Drag } (C_{D_{\text{min}}}) &\equiv& \text{Zero-lift Drag } (C_{D_0}) &=& \text{Zero-lift drag from} & \text{[Profile]}\\
&& \text{Induced Drag } (C_{D_i}) &=& \text{Induced drag from} & \text{[Profile + Lift-Induced Drag]}
\end{eqnarray}

*Author's note: I actually don't know if people incorporate elements of wave drag into the above model, or if wave drag is another discrete term that is simply added on. I've only ever seen model 2 defined with these terms (because general aviation literature tends to be written for subsonic application).*

## Estimation during Conceptual Analysis

#### Subsonic Flight

The first model contains a more convenient mathematical breakdown of drag into component terms, whereas for the napkin math typical of subsonic vehicle conceptual analysis, the second model is more convenient to wield.
For example, it is not uncommon to see $C_{D_0} (= C_{D_{\text{min}}}) = \text{constant}$ and $C_{D_i} = f(C_L)$ (induced drag is a function of the lift coefficient).
Since induced drag is approximately quadratically related to $C_L$, the full drag coefficient $C_D$ is conveniently represented with a polynomial.

So how in practice does one compute $C_{D_i}$ in model 2 for a subsonic case?
It turns out that after exclusion of wave drag from the models, the $C_{D_i}$ expression of model 1 can be modified to include the lift-dependent profile drag.
Presently, the first model defines lift-induced drag as

$$
C_{D_{i}}
\equiv \frac{{C_L}^2}{\pi e \text{AR}}
= k \cdot {C_L}^2
\quad \text{with} \quad
k
= \frac{1}{\pi e \text{AR}}
$$

where the induced drag coefficient $C_{D_{i}}$ is proportional to the lift coefficient $C_L$ by the induced drag factor $k$ - which itself is a function of the span efficiency $e$ and the effective aspect ratio of the wing $\text{AR}$.
An appealing ansatz is to take the lift-induced contribution of profile drag as a function of the lift coefficient squared, in which case this drag component is conveniently absorbed into the induced drag formulation

$$
C_{D_{i,\text{mod}}}
= (m + k) \cdot {C_{L}}^2
\quad \text{with} \quad
m + k
= \frac{1}{\pi e_0 \text{AR}}
$$

where the span efficiency $e$ has been replaced with the Oswald efficiency factor $e_0$.
While $m$ is not often given in literature, it's quite easy to find $e_0$ (or even guess-timate it as a slightly less efficient $e$).

$$
\frac{1}{\pi e_0 \text{AR}}
= m + \frac{1}{\pi e \text{AR}}
\implies
\frac{1}{e_0}
= \frac{1}{e} + \pi \text{AR} \cdot m
$$

$$
e_0 \lt e
\quad \text{since} \quad
m \gt 0
$$

#### Transonic/Supersonic Flight

In *Aircraft Design: A Conceptual Approach* by Daniel P. Raymer, the author describes a method known as the leading-edge-suction method (LESM) to estimate the value of the Oswald efficiency during trans- and supersonic flight regimes.

A description of LESM for estimating $e_0$ in high-speed regimes is **not yet** included here.
This is not to suggest that the results from LESM aren't reasonable, I'm just not looking at trans-/supersonic flight at this time.

## Profile Drag, $C_{D_0}$

Profile drag can largely be thought of as the drag associated with the 2D profile of aerofoils.
Profile drag is also sometimes known as *parasitic drag* (an ever present drag component sapping away at your aircraft's hard earned energy state).

The proportion of profile drag made up of skin friction drag for aerodynamic shapes tends to vary between 80-100% of the total profile drag.

#### *Zero-lift* component

- Pressure drag, a.k.a Form drag

Pressure (form) drag is the drag associated with the separation of flow from a surface.
The resulting turbulence and eddy currents in the wake flow of an object are responsible for a suction-type drag effect.
Form drag is a function of how slender or streamlined an object is.

In addition to this, the mixing of dissimilar airflows causes *interference* drag.
This is a type of form drag in which turbulence and eddy currents in the wake flow of an object, as you might've guessed by now, *also* create a suction-type drag effect.

Air molecules do not care that they are moving around a weapons pylon, a wing, control surface, or nose gear - the molecules encounter "*the vehicle*" just the same - some flow separates around some geometries, and other flows mix poorly around others (both causing form drag).

- Skin friction drag

Thanks to the *no-slip boundary condition*, particles in viscous fluids closest to a solid boundary have no velocity relative to the boundary - i.e. the forces of adhesion to the wall are stronger than the forces of cohesion with other particles closer to the freestream.
The size of the boundary layer, how laminar or turbulent it is, and the viscosity of the fluid all play important roles in being able to estimate the frictional drag force caused by the aerodynamically *wetted* skin of the object.

Skin friction drag can be estimated by considering a coefficient of friction somehwere between empirically determined laminar and turbulent solutions.
The position at which oncoming laminar flow transitions to turbulent flow is dictated by the flow Reynolds number, as well as the quality of the aerodynamic surface's finish.

#### *Lift-induced* component

- Skin friction drag

In addition to the smoothness of the aerodynamic surface and Reynolds number, the transition point of a flow from laminar to turbulent is also given by the pressure gradient over an aerofoil (which changes with angle of attack).
A favourable pressure gradient is that in which the pressure gradient is reducing (negative gradient) causing the flow to accelerate and stabilise the boundary layer, allowing for an extensive laminar regime.
Giving the flow too short a distance to decelerate will cause flow separation and increased effective skin friction.
Varying the sectional angle of attack will change the chordwise pressure distribution, impacting transition points on upper and lower surfaces.

- Pressure drag

The effect of flow separation is an undesirable pressure recovery on the low pressure (upper) surface of a lifting body.
For a wing, this resulting reduction in pressure differential between upper and lower surfaces reduces the total lift force capacity of the wing.

## Induced Drag, $C_{D_i}$

The nomenclature can be a little confusing.
In this context, induced drag primarily refers to the drag produced explicitly due to lift-induced changes in the local angle of attack of a wing (thanks to variable pressure distributions along a wing's span).
This term does not include lift-induced drag subcomponents of Profile or Wave drag.

*By definition, this term has no zero-lift drag component.*

#### *Lift-induced* component

- Pressure drag, a.k.a Vortex drag

Induced drag is a purely 3D phenomena, in that it is caused by the lateral motion of air across a wing.
The higher pressure surfaces below a wing spill pressure energy around the wing tips into the lower pressure zones above a wing.
This causes a swirling vortex of air with an axis of rotation near the wing tips (hence they are called 'tip vortices'), where air below the wing moves outboard, and air above the wing moves inward.
This movement of air is known as 'downwash', induced by the tip vortices.

According to Prandtl's Lifting-line Theory (PLLT), lifting surfaces can be modelled as a continuous sheet of vortices and aptly models this circulatory flow.
Considering for a moment a 2D section of the wing - the downwash induces a local, downward velocity component that reduces the 
section's effective angle of attack $\alpha_{\text{eff}}$ by the induced angle of attack $\alpha_i$.

$$\alpha_{\text{eff}} = \alpha_{\infty} - \alpha_i$$

Despite freestream angle of attack $\alpha_{\infty}$ remaining unchanged, the local reduction in a section's angle of attack has the effect of tilting the lift vector backwards (towards the trailing edge) by $\alpha_i$.
This rotation of this local, sectional lift vector means less of it is contributing to the wing's lift force normal to the freestream direction we care about - and even worse, the remaining component of the vector now acts in a direction opposing motion of the wing (hence 'lift-induced' drag).

## Wave Drag, $C_{D_w}$  

\**Section coming soon*\*

#### *Zero-lift* component

- Wave drag due to volume

#### *Lift-induced* component

- Wave drag due to lift